# 3.3. Concise Implementation of Linear Regression
## 3.3.1. Generating the Dataset

In [1]:
import numpy as np
import torch
from torch.utils import data
import random

def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

## 3.3.2. Reading the Dataset

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[ 0.9863,  1.3693],
         [ 0.3436,  0.1897],
         [-1.3155, -0.2248],
         [-0.6406,  1.3335],
         [ 0.2750,  0.7004],
         [-0.6155, -0.6138],
         [-0.6604, -0.2653],
         [ 0.2938, -0.9910],
         [ 0.3019, -0.4333],
         [-0.0625,  0.5279]]),
 tensor([[ 1.5142],
         [ 4.2378],
         [ 2.3287],
         [-1.6103],
         [ 2.3606],
         [ 5.0574],
         [ 3.7809],
         [ 8.1541],
         [ 6.2667],
         [ 2.2771]])]

## 3.3.3. Defining the Model

In [3]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 3.3.4. Initializing Model Parameters

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 3.3.5. Defining the Loss Function

In [5]:
loss = nn.MSELoss()

## 3.3.6. Defining the Optimization Algorithm

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 3.3.7. Training

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000228
epoch 2, loss 0.000098
epoch 3, loss 0.000098


In [8]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0002, 0.0005])
error in estimating b: tensor([0.0007])
